In [13]:
%ENV
TIMESTAMPING=1
BLOCKCHAIN_LOGGING=1
LOGGING_LEVEL=10

SET ENV TIMESTAMPING=1
SET ENV BLOCKCHAIN_LOGGING=1
SET ENV LOGGING_LEVEL=10

In [2]:
%file: foobar.pl

man(socrates).
man(hayden).
mortal(X) :- man(X).

File: foobar.pl
Timestamp: 1638916047564098865
Hash: 7d8e47a0445000e3acbd1806184c16f9
Domain: swipluser-foobar
Iroha Response: COMMITTED

In [4]:
?- mortal(socrates).
?- mortal(X).
?- mortal(bob).

true.
X = socrates ;
X = hayden .
false.

In [15]:
%python
print("Hello, World!")

Hello, World!

![Image](images/prolog-image.png)